In [9]:
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


from selenium.webdriver.support.wait import WebDriverWait

from datetime import datetime
import pandas as pd
import numpy as np
import random
import time
import os
import re


In [12]:
now = datetime.now()

start_date = now.strftime('%m%d%Y')
# make data/playlist/ directory
os.makedirs('jobstreet/' + start_date , exist_ok=True)
chrome_path = r'C:/Users/user/scraper_sel/chromedriver.exe' # This is the location of the chronedriver on my local machine

service = Service(executable_path=chrome_path)  

chrome_options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=service)


url = r'http://www.jobstreet.com.ph/'

driver.get(url)
driver.implicitly_wait(8)


keyword = 'Data Science'
location = 'Philippines'

keyword_field = driver.find_element(By.ID, 'searchKeywordsField')
keyword_field.clear()
keyword_field.send_keys(keyword)

location_field = driver.find_element(By.ID, 'locationAutoSuggest')
location_field.clear()
location_field.send_keys(location)

find = driver.find_element(By.XPATH, "//button[@type='submit']")
find.click()



In [13]:
page = 1

while True

    job_post_index = 0

    try:

        print(f'Harvesting page: {page}....', end="")

        articles = driver.find_elements(By.TAG_NAME, "article")

        time.sleep(1 + 2*random.random())
        to_frame = []
        items = 0

        for article in articles:
        
            time.sleep(1 + 2*random.random())

            job_button = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable(article))

            try:

                driver.execute_script('arguments[0].click();', job_button)
                items += 1

                try:
                    summary = driver.find_element(By.XPATH, "//div[@data-automation='jobDescription']").text.strip()

                except:
                    summary = None
                
            except:
                print(f'click {job_post_index} failed')

            location = article.find_elements(By.TAG_NAME, 'span')[3].text.strip()
            title = article.find_elements(By.TAG_NAME, 'span')[0].text.strip()
            company = article.find_elements(By.TAG_NAME, "span")[1].text.strip()

            test_salary = article.find_elements(By.TAG_NAME, "span")[5].text.strip()

            if re.search('php', test_salary.lower()):
                salary = test_salary
            else:
                salary = None

            # try to get url,
            job_url = article.find_elements(By.TAG_NAME, 'a')[0].get_attribute('href')
         
            job_info = {
                'location': location,
                'title': title,
                'company': company,
                'salary': salary,
                'summary': summary,
                'job_url': job_url}

            to_frame.append(job_info)
            job_post_index += 1
        
        framed_df = pd.DataFrame(to_frame)
        framed_df.to_csv('jobstreet/' + start_date + '/page' + str(page) + '.csv', index=False)
        print('Saved!')

        start_url = driver.current_url

        next_page = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH,
                "//span[normalize-space()='Next']"
                )))

        driver.execute_script("arguments[0].click();", next_page)

        next_url = driver.current_url

        if start_url == next_url:
            print(f"Scraper stopped moving, scaping ended at page {page - 1}")
            break

        page += 1

    except:
        print(f"Exception hit, scaping ended at page {page - 1}")
        break

print('/nScrape End')



Harvesting page: 1....Saved!
Harvesting page: 2....Saved!


In [ ]:
# combines all scraped files into 1 raw file

file_count = len(os.listdir("./jobstreet/" + start_date ))

result_df = pd.read_csv('jobstreet/' + start_date +'/page1.csv')

for page in range(2,file_count,1):

    print(f"loading page {page}....", end="")

    df_to_add = pd.read_csv('jobstreet/' + start_date + '/page'+ str(page) + '.csv')

    result_df = pd.concat([result_df, df_to_add])

    print('Done!')

result_df.to_csv('jobstreet/raw_' + start_date + '/page' + str(page) + '.csv', index=False)

result_df.shape